# BM25 baseline

This is an evaluation based on BM25 as baseline.
The evaluations are performed on WITSML dataset and a public dataset [clinc_oos](https://huggingface.co/datasets/clinc/clinc_oos).


## Evaluation on WITSML


In [11]:
import math
import re
import numpy as np
from collections import Counter

# -----------------------------
# Simple, dependency-free BM25
# -----------------------------


def tokenize(s: str):
    _word_re = re.compile(r"[A-Za-z0-9%/±]+")
    return [t.lower() for t in _word_re.findall(s or "")]


def concat_with_weights(fields, weights):
    """
    fields: dict[str, list[str] | str]  (already tokenized or raw)
    weights: dict[str, float]
    returns: list[str] tokens with repetition to reflect weights
    """
    bag = []
    for fname, content in fields.items():
        w = weights.get(fname, 1.0)
        if content is None:
            continue
        if isinstance(content, str):
            toks = tokenize(content)
        else:
            # list[str]
            toks = []
            for x in content:
                toks.extend(tokenize(x))
        # repeat tokens by weight's integer part, keep fractional via probabilistic drop
        k = int(w)
        frac = w - k
        if k > 0:
            bag.extend(toks * k)
        if frac > 0:
            # lightweight fractional weighting via sampling-free rounding
            # (multiply some tokens by +1)
            extra = round(frac * len(toks))
            bag.extend(toks[:extra])
    return bag


class BM25:
    def __init__(self, docs, k1=1.5, b=0.75):
        """
        docs: list[list[str]] tokenized docs
        """
        self.k1 = k1
        self.b = b
        self.docs = docs
        self.N = len(docs)
        self.doc_lens = [len(d) for d in docs]
        self.avgdl = sum(self.doc_lens) / max(1, self.N)
        # df and idf
        self.df = Counter()
        for d in docs:
            for t in set(d):
                self.df[t] += 1
        # Using BM25+ style log idf (Okapi variant)
        self.idf = {t: math.log(1 + (self.N - df + 0.5) / (df + 0.5)) for t, df in self.df.items()}
        # term frequencies
        self.tf = [Counter(d) for d in docs]

    def score(self, q_tokens):
        scores = [0.0] * self.N
        for i in range(self.N):
            dl = self.doc_lens[i]
            denom_norm = self.k1 * (1 - self.b + self.b * dl / max(1e-9, self.avgdl))
            tf_i = self.tf[i]
            s = 0.0
            for t in q_tokens:
                if t not in self.idf:
                    continue
                f = tf_i.get(t, 0)
                if f == 0:
                    continue
                s += self.idf[t] * (f * (self.k1 + 1)) / (f + denom_norm)
            scores[i] = s
        return scores


def bm25_predict(sample_dict, labels_dict, build_query_tokens_func, build_label_doc_func, k1=1.5, b=0.75, top_k=5):
    label_names = list(labels_dict.keys())
    docs = [build_label_doc_func(labels_dict[name]) for name in label_names]
    model = BM25(docs, k1=k1, b=b)
    q = build_query_tokens_func(sample_dict)
    scores = model.score(q)
    ranked = sorted(zip(label_names, scores), key=lambda x: x[1], reverse=True)
    result = {}
    for i in range(top_k):
        result.update({ranked[i][0]: ranked[i][1]})
    return result


def softmax_dict(d: dict) -> tuple:
    # Step 1: softmax
    values = np.array(list(d.values()), dtype=float)
    exp_values = np.exp(values - np.max(values))
    softmax_values = exp_values / np.sum(exp_values)
    softmaxed = dict(zip(d.keys(), softmax_values))

    # Step 2:return key with max
    max_val = np.max(softmax_values)
    for k, v in softmaxed.items():
        if np.isclose(v, max_val):
            max_key = k
            break
    return softmaxed, max_key

Turning your schema into BM25 I/O. Choose simple, sensible field weights (tweak as you like).

Create `build_query_tokens` and `build_label_doc` for prototyepedata, unit, quantity respectively.


In [12]:
def build_query_tokens(sample):
    QUERY_FIELD_WEIGHTS = {
        "Mnemonic": 4.0,
        "Description": 3.0,
        "Unit": 2.0,  # units are helpful signals (e.g., bar, m/s)
    }
    fields = {
        "Mnemonic": sample.get("Mnemonic", ""),
        "Description": sample.get("Description", ""),
        "Unit": sample.get("Unit", ""),
    }
    return concat_with_weights(fields, QUERY_FIELD_WEIGHTS)


def build_label_doc_prototypedata(label_meta):
    LABEL_FIELD_WEIGHTS = {
        "mnemonic": 4.0,  # exact code like "SPP", "ROP" is very telling
        "comment": 3.0,  # human description
        "base_quantities": 1.5,  # PressureQuantity, VelocityQuantity, etc.
        "measurable_quantity": 1.0,
        "common_mnemonics": 4.0,  # other aliases/codes
    }
    fields = {
        "mnemonic": label_meta.get("ddhub:PrototypeData"),
        "comment": " ".join(label_meta.get("rdfs:comment", [])),
        "base_quantities": label_meta.get("ddhub:IsOfBaseQuantity", []),
        "measurable_quantity": label_meta.get("ddhub:IsOfMeasurableQuantity"),
        "common_mnemonics": label_meta.get("zzz:commonMnemonics", []),
    }
    return concat_with_weights(fields, LABEL_FIELD_WEIGHTS)


def build_label_doc_quantity(label_meta):
    LABEL_FIELD_WEIGHTS = {
        "mnemonic": 4.0,  # exact code like "SPP", "ROP" is very telling
        "comment": 3.0,  # human description
        "quantityHasUnit": 2.0,  # PressureQuantity, VelocityQuantity, etc.
        "prototypeData": 1.0,
    }
    fields = {
        "mnemonic": label_meta.get("ddhub:Quantity"),
        "comment": " ".join(label_meta.get("rdfs:comment", [])),
        "quantityHasUnit": label_meta.get("zzz:QuantityHasUnit", []),
        "prototypeData": label_meta.get("zzz:PrototypeData", []),
    }
    return concat_with_weights(fields, LABEL_FIELD_WEIGHTS)


def build_label_doc_unit(label_meta):
    LABEL_FIELD_WEIGHTS = {
        "mnemonic": 4.0,  # exact code like "SPP", "ROP" is very telling
        "comment": 3.0,  # human description
        "isUnitForQuantity": 1.5,  # PressureQuantity, VelocityQuantity, etc.
        "common_mnemonics": 4.0,  # other aliases/codes
    }
    fields = {
        "mnemonic": label_meta.get("ddhub:PrototypeData"),
        "comment": " ".join(label_meta.get("rdfs:comment", [])),
        "isUnitForQuantity": label_meta.get("ddhub:IsUnitForQuantity", []),
        "common_mnemonics": label_meta.get("zzz:commonMnemonics", []),
    }
    return concat_with_weights(fields, LABEL_FIELD_WEIGHTS)


In [13]:
# -----------------------
# Minimal working example
# -----------------------

sample = {"Mnemonic": "SPP", "Description": "SPP is a stand pipe pressure. It has a known accuracy of approx. ±5%.", "Unit": "bar"}

labels_obj = {
    "ROP": {
        "ddhub:PrototypeData": "ROP",
        "rdfs:comment": ["Rate of penetration, the speed at which the drill bit advances."],
        "ddhub:IsOfMeasurableQuantity": "RateOfPenetrationDrillingQuantity",
        "ddhub:IsOfBaseQuantity": ["UncertainQuantity", "OutOfSetQuantity", "VelocityQuantity"],
        "zzz:commonMnemonics": ["ROP"],
    },
    "SPP": {
        "ddhub:PrototypeData": "SPP",
        "rdfs:comment": ["Standpipe pressure, the pressure in the drill pipe."],
        "ddhub:IsOfMeasurableQuantity": "PressureDrillingQuantity",
        "ddhub:IsOfBaseQuantity": ["UncertainQuantity", "OutOfSetQuantity", "PressureQuantity"],
        "zzz:commonMnemonics": ["SPP"],
    },
}

recognition_ranking = bm25_predict(
    sample_dict=sample,
    labels_dict=labels_obj,
    build_query_tokens_func=build_query_tokens,
    build_label_doc_func=build_label_doc_prototypedata,
    top_k=2,
)
print(recognition_ranking)
# Expected: [('SPP', <bigger>), ('ROP', <smaller>)]

{'SPP': 18.130479576577475, 'ROP': 3.3763616245040318}


Now, run the recognition on real WITSML metadata.


In [14]:
from pathlib import Path
import json
import sys

current_dir = Path().resolve()
project_root = current_dir.parent

sys.path.append(str(project_root))

import server.utils.configs.globals_config as glb  # noqa
import server.utils.filter as ft  # noqa

# Kick out zzz#TBD, OutOfSet, Uncertain
kick_out_keys = ["zzz#TBD", "OutOfSetPrototypeData", "UncertainPrototypeData", "OutOfSetQuantity", "UncertainQuantity", "OutOfSetUnit", "UncertainUnit"]
for k in kick_out_keys:
    glb.quantity_fullList_extraContent.pop(k, None)
    glb.unit_fullList_extraContent.pop(k, None)
    glb.prototypeData_fullList_extraContent.pop(k, None)

query_files = [
    project_root / "data_store/test_data/new_labels/extracted_json/9-F-9_A+1+log+1+1+1+00001.json",
    project_root / "data_store/test_data/new_labels/extracted_json/9-F-9_A+1+log+2+1+1+00001.json",
    project_root / "data_store/test_data/new_labels/extracted_json/9-F-9_A+1+log+2+2+1+00001.json",
]

query_objs = []
for i in range(len(query_files)):
    with open(query_files[i], "r", encoding="utf-8") as f:
        data = json.load(f)
        query_tmp = {}
        for j in range(len(data)):
            mnemonic = data[j]["Mnemonic"]
            data[j].pop("true label PrototypeData")
            data[j].pop("true label Unit")
            data[j].pop("true label Quantity")
            query_tmp.update({mnemonic: data[j]})
        query_objs.append(query_tmp)
        filtered, rejected = ft.filter_metadata(query_objs[i])
        print("filtered count:", len(filtered))
        print("rejected count:", len(rejected))

    recognition_results = {}
    for k, v in filtered.items():
        result_quantity = bm25_predict(
            sample_dict=v,
            labels_dict=glb.quantity_fullList_extraContent,
            build_query_tokens_func=build_query_tokens,
            build_label_doc_func=build_label_doc_quantity,
            top_k=5,
        )
        result_unit = bm25_predict(
            sample_dict=v,
            labels_dict=glb.unit_fullList_extraContent,
            build_query_tokens_func=build_query_tokens,
            build_label_doc_func=build_label_doc_unit,
            top_k=5,
        )
        result_prototypeData = bm25_predict(
            sample_dict=v,
            labels_dict=glb.prototypeData_fullList_extraContent,
            build_query_tokens_func=build_query_tokens,
            build_label_doc_func=build_label_doc_prototypedata,
            top_k=5,
        )
        recognition_results.update(
            {
                k: {
                    "Raw_content": str(v),
                    "Quantity_class": softmax_dict(result_quantity)[1],
                    "Quantity_class_candidates": softmax_dict(result_quantity)[0],
                    "Unit_class": softmax_dict(result_unit)[1],
                    "Unit_class_candidates": softmax_dict(result_unit)[0],
                    "PrototypeData_class": softmax_dict(result_prototypeData)[1],
                    "PrototypeData_class_candidates": softmax_dict(result_prototypeData)[0],
                }
            }
        )

    Path("BM25").mkdir(parents=True, exist_ok=True)
    with open(f"./BM25/recognition_results_{i + 1}.json", "w", encoding="utf-8") as f:
        json.dump(recognition_results, f, ensure_ascii=False, indent=2)


filtered count: 189
rejected count: 9
filtered count: 98
rejected count: 1
filtered count: 109
rejected count: 1


## Same evaluation on the public dataset clinc_oos

Evaluate the non-ID and ID classification performace using a public dataset [clinc_oos](https://huggingface.co/datasets/clinc/clinc_oos).
For simplication purpose, only test on the small/validation set, which including 3100 samples.


In [15]:
# Load dataset
import pandas as pd
import yaml


dataset_path = project_root / "data_store/test_data/clinc_oos/small/reduced_validation_set.parquet"  # path to reduced dataset
df = pd.read_parquet(dataset_path)
true_label_dict = dict(zip(df["text"], df["intent"]))
print(true_label_dict)

labels_path = project_root / "data_store/test_data/clinc_oos/labels.yaml"
with open(labels_path, "r") as file:
    labels_obj = yaml.safe_load(file)
full_labels = labels_obj.get("names")

labels = {str(i): full_labels[str(i)] for i in range(30)}
labels.update({"42": "oos"})

bm25_doc, label_names = [], []
for k, v in labels.items():
    str_list = v.split("_")
    bm25_doc.append(str_list)
    label_names.append(v)
print("bm25_doc", bm25_doc)


# Simple tokenizer (lowercase + split on non-alphabetic chars)
def simple_tokenizer(text):
    return re.findall(r"\b\w+\b", text.lower())


# Apply tokenizer to the "text" column
tokenized_corpus = df["text"].apply(simple_tokenizer).tolist()
# print(tokenized_corpus[:5])  # show first 5


def bm25_predict_for_clinc(tokenized_corpus: list, top_k: int = 5):
    model = BM25(bm25_doc)
    scores = model.score(tokenized_corpus)
    ranked = sorted(zip(label_names, scores), key=lambda x: x[1], reverse=True)

    result = {}
    for i in range(top_k):
        result.update({ranked[i][0]: ranked[i][1]})
    return result


result_clinc = {}
for i in range(len(tokenized_corpus)):
    q_str = df["text"].values[i]
    intent = str(true_label_dict[q_str])
    tmp = {"true_label": labels[intent], "candidates": bm25_predict_for_clinc(q_str)}
    result_clinc.update({q_str: tmp})
print(result_clinc)

Path("BM25").mkdir(parents=True, exist_ok=True)
with open("./BM25/recognition_results_clinc.json", "w", encoding="utf-8") as f:
    json.dump(result_clinc, f, ensure_ascii=False, indent=2)

{"does mcdonald's have good reviews": '0', "are the review for mcdonald's good or bad": '0', "how are the reviews for mcdonald's": '0', "what are the review for mcdonald's": '0', "how are those mcdonald's reviews looking": '0', 'what are the ratings for yardhouse': '0', 'do you know how the ratings are for yardhouse': '0', 'are the ratings for yardhouse good': '0', 'can you tell me the ratings for yardhouse': '0', 'how good are the review for yardhouse': '0', "how healthy is shepard's pie": '1', 'how healthy is pecan pie': '1', 'how healthy is hummus': '1', 'how healthy is salad': '1', 'how healthy is pumpkin pie': '1', 'is chocolate good for you': '1', 'tell me how health chocolate is': '1', "what is the nutritional info for kraft mac 'n' cheese": '1', 'is chocolate bad for you or good for you': '1', 'can you tell me the nutrional info of a big mac': '1', 'my account looks to be blocked and i have no idea why': '2', "it seems that my account is blocked and i'm not sure why": '2', "my 